In [9]:
import pandas as pd
import string
from nltk.corpus import wordnet as wn

# Esercizio 1.5 - Esperimento content-to-form

Questo esercizio è di tipo __content-to-form__ ovvero partiamo dal contenuto (il significato) e vogliamo arrivare alla forma (la parola).
Il nostro obiettivo è quello di ottenere il synset di WordNet corretto per le quattro parole del file _defs.csv_ a partire dalle definizioni in esso presenti.

Carichiamo il file con le definizioni dei termini.

In [10]:
df_defs = pd.read_csv(filepath_or_buffer="utils/defs.csv", index_col=0).dropna()
df_defs.reset_index(drop=True, inplace=True)
df_defs.head()

,Courage,Paper,Apprehension,Sharpener
0,property that allows you to face any situation...,"cellulose material that can be cut, folded and...","something strange, which causes a strange feel...",tool equipped with a blade that allows you to ...
1,Ability to face our own fears and do something...,Material derived from trees and used in sever...,fearful expectation or anticipation,Object used to shapen a pencil
2,the ability to face thing without fear,a type of material made from cellulose,A mood where one feel agitation,An object to sharpen a pencil
3,Inner strength that allow you to face particul...,Product obtained from wood cellulose. It is us...,State of disturbance,Tool used to sharpen pencils
4,Ability to control the fear,Flat material made from wood used for writing,Worry about the future,Little object which allow to sharpen a pencil


Effettuiamo preprocessing per avere risultati migliori.

In [11]:
# Loading stopwords list from file
stopwords = []
for line in open("utils/stop_words_FULL.txt", 'r').readlines():
    stopwords.append(line.rstrip('\n'))
stopwords = pd.Series(stopwords)

for c in df_defs.columns:
    # Lowercasing
    df_defs[c] = df_defs[c].str.lower()
    # Punct removal
    tmp = df_defs[c].apply(lambda x: str(x).translate(str.maketrans('', '', string.punctuation)))
    df_defs[c] = tmp
    # Stopword removal
    tmp = df_defs[c].apply(lambda x: ' '.join([word for word in str(x).split() if word not in stopwords.values]))
    df_defs[c] = tmp
df_defs.head()

,Courage,Paper,Apprehension,Sharpener
0,property allows face situation despite feeling...,cellulose material cut folded written,strange strange feeling strangeness normal abn...,tool equipped blade allows sharpen pencils
1,ability face fears scars unpleasant,material derived trees context,fearful expectation anticipation,object shapen pencil
2,ability face thing fear,type material cellulose,mood feel agitation,object sharpen pencil
3,inner strength allow face situations,product wood cellulose writing,disturbance,tool sharpen pencils
4,ability control fear,flat material wood writing,worry future,object allow sharpen pencil


Siccome in WordNet sono presenti molti synsets sfruttiamo il meccanismo del __genus-differentia__ per ridurre lo spazio di ricerca.
Il _genus-differentia_ usa un iperonimo per individuare la categoria della parola (_genus_) e le sue proprietà per distinguerla dagli altri elementi (_differentia_).

Dalle definizioni prendiamo le 4 parole più frequentemente utilizzate, ognuna di esse rappresenta un candidato _genus_.

In [12]:
def get_frequent_words(df, col):
    """
    Function to get first 4 frequent words.

    :param df: dataframe with sentences
    :param col: column to search in

    :return: first 4 frequent words in the given column
    """
    sents = []
    df[col].apply(lambda x: sents.append(x.split()))
    words = []
    # costruiamo una lista di parole
    for s in sents:
        words += s
    # contiamo le parole in un dizionario
    counts = dict.fromkeys(words, 0) # inizializziamo i conteggi di tutte le parole a 0
    for w in words:
        counts[w] += 1
    # ordiniamo le parole in ordine decrescente
    df2 = pd.DataFrame(data={'word': counts.keys(), 'frequency': counts.values()})
    df2.sort_values(by='frequency', ascending=False, inplace=True)
    return df2[:4] # prendiamo solo le prime 4 più frequenti

In [13]:
freq_words = dict.fromkeys(df_defs.columns)
for col in df_defs.columns:
    freq_words[col] = get_frequent_words(df_defs, col)
    print('\n---------', col, '---------')
    print(freq_words[col])


--------- Courage ---------
          word  frequency
7      ability         19
6         fear         14
2         face          9
15  situations          6

--------- Paper ---------
         word  frequency
1    material         22
18      write          8
11    writing          8
0   cellulose          7

--------- Apprehension ---------
       word  frequency
24     fear         10
32  anxiety         10
1   feeling          6
36   happen          5

--------- Sharpener ---------
      word  frequency
0     tool         16
4  sharpen         16
8   pencil         16
6   object         12


Passiamo dalle parole frequenti ai loro synset accedendo a WN.

In [14]:
for k in freq_words.keys():
    df = freq_words[k]
    df['syn'] = df['word'].apply(lambda x: wn.synsets(x))
    freq_words[k] = df
    print('\n---------', k, '---------')
    print(freq_words[k])


--------- Courage ---------
          word  frequency                                                syn
7      ability         19   [Synset('ability.n.01'), Synset('ability.n.02')]
6         fear         14  [Synset('fear.n.01'), Synset('concern.n.02'), ...
2         face          9  [Synset('face.n.01'), Synset('expression.n.01'...
15  situations          6  [Synset('situation.n.01'), Synset('situation.n...

--------- Paper ---------
         word  frequency                                                syn
1    material         22  [Synset('material.n.01'), Synset('material.n.0...
18      write          8  [Synset('write.v.01'), Synset('write.v.02'), S...
11    writing          8  [Synset('writing.n.01'), Synset('writing.n.02'...
0   cellulose          7                         [Synset('cellulose.n.01')]

--------- Apprehension ---------
       word  frequency                                                syn
24     fear         10  [Synset('fear.n.01'), Synset('concern.n.02'), .

Per ogni synset trovato guardiamo fra i suoi iponimi (essendo il genus un iperonimo) alla ricerca del synset corretto per le parole target. Valutiamo l'intersezione fra la definizione del synset e quelle del file.

In [15]:
NUM_DEFS = len(df_defs) # 30

def bag_of_words(d, defs):
    """
    Find the average intersection length between a definition and the others.

    :param d: the definition
    :param defs: the other definitions

    :return: average length of the intersection
    """
    d = set(d.split())
    sum = 0
    for other_d in defs:
        bow = d.intersection(other_d.split()) # calcolo l'intersezione fra le due definizioni
        sum += len(bow) / min(len(d), len(other_d.split())) # normalizzazione
    return sum/NUM_DEFS

In [16]:
for k in freq_words.keys(): # per ogni parola target
    df = freq_words[k] # prendiamo le sue parole frequenti
    best = (None, 0)
    for syn in df['syn']: # per ogni synset (rappresentante un candidato genus)
        nodes = syn
        while nodes:
            syn = nodes[0]
            d = syn.definition()
            bow_score = bag_of_words(d, df_defs[k]) #todo controllare se va bene usare bag_of_words() oppure se serve l'altra funzione compute_sim() del lab_1.1
            if bow_score > best[1]:
                best = (syn, bow_score)
            nodes.remove(syn) # per evitare loop
            nodes += syn.hyponyms() # esploriamo tutta la porzione di albero di WN con radice il candidato genus fino alle foglie
    print('---------', k, '---------')
    print(best)
    print('syn definition: ', best[0].definition(), end='\n\n')


--------- Courage ---------
(Synset('hand.n.04'), 0.6333333333333333)
syn definition:  ability

--------- Paper ---------
(Synset('paper.n.01'), 0.3544444444444445)
syn definition:  a material made of cellulose pulp derived mainly from wood or rags or certain grasses

--------- Apprehension ---------
(Synset('panic.n.01'), 0.24222222222222223)
syn definition:  an overwhelming feeling of fear and anxiety

--------- Sharpener ---------
(Synset('jaw.n.03'), 0.2883333333333333)
syn definition:  holding device consisting of one or both of the opposing parts of a tool that close to hold an object



Come possiamo notare il sistema è riuscito ad individuare il senso corretto per la parola _Paper_, questo era ipotizzabile dal momento che si tratta di un oggetto concreto, con features ben definite e quindi facilmente individuabili.

Riguardo la parola _Apprehension_ abbiamo ottenuto un risultato affine a quello corretto, qui si tratta però di uno oggetto astratto, per il quale è difficile ottenere una descrizione accurata, quindi il sistema ha comunque ottenuto un buon risultato. Inoltre, guardando alle parole frequenti delle definizioni del file, notiamo che 3/4 di esse sono state usate nella definizione del synset, sintomo di una corretta esecuzione.

Discorso diverso per la parola _Courage_, che è anch'essa un oggetto astratto ma per la quale i risultati non sono quelli sperati. Di nuovo notiamo come l'unica parola della definizione del synset sia la parola maggiormente utilizzata nelle definizioni del file, questo può essere il motivo di tale risultato.

In conclusione anche per _Sharpener_ otteniamo un risultato errato, qui però l'oggetto è fisico e quindi il sistema avrebbe dovuto trovare il senso corretto. Una possibile causa di questo errore è data dai synset candidati genus, infatti alcuni di essi sono verbi, i quali vengono rappresentati in alberi separati all'interno di WN, tagliando quindi dalla ricerca il senso corretto (ad esempio i synsets di _sharpen_ nelle parole frequenti).